# MDBA Machine Learning Notebook

Annotating and training an object detector for automated counting of species

IMPORTANT NOTE: Parameters in params.yaml are used throughout the notebook.

The paths used throughout the notebook are relative and therefore the working directory must be the root of the machine learning directory. E.g.

<ul>
<li>Root</li>
    <ul>
    <li>Data</li>
    <li>Models</li>
    <li>Scripts</li>
    <li>Utils</li>
    <li>requirements.txt</li>
    <li>params.yaml</li>
    <li>README.pdf</li>
    <li>README.md</li>
    </ul>
</ul>


In [3]:
import os
# Set working directory here
os.chdir(r'D:\20220600_AIML_BirdCounting')
os.getcwd()

'D:\\20220600_AIML_BirdCounting'

In [4]:

import cv2
import json
import yaml
import pylab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from pathlib import Path
from random import seed, shuffle, sample
from shutil import copyfile, move
from subprocess import run, Popen
from yaml import safe_load

from utils.Slicer import Slicer
from utils.utils import ensure_path
from utils.data_utils import CocoDataset
from utils.utils import str2bool
from utils.count import count_folder
from utils.train_utils import get_validation_transform

In [5]:
with open('./params.yaml', 'r') as params_file:
        params = yaml.safe_load(params_file)

# Split raw dataset 

This module takes raw images and a point file created in dotdotgoose to split them into appropriate size slices for training.

Slice size, as well as raw image size must be specified in the .yaml read in in the 2nd code chunk.

The slices are automatically placed into a new directory to be ingested by the 'prepare_training_set.py' module

### Function definitions

In [6]:
def _save_slices(slice_dict, input_dict, input_dir, output_dir, datatype):
    """
    This function takes a list of inputs, and for each input does the following:
        - Reads the file as an image
        - For each slice in slice_set, crops the image and saves the output
    """    
    # Generate and save slices for each file in input dictionary
    output_dir = ensure_path(output_dir)
    for filename, slice_set in tqdm(input_dict.items(), desc=f"Saving {datatype} slices"):
        img = cv2.imread(str(input_dir/filename))
        for slice_id in slice_set:
            l,t,r,b = slice_dict[slice_id]
            slice = img[t:b, l:r]
            slicename = (os.path.splitext(filename)[0] + f"_t{t:04d}_b{b:04d}_l{l:04d}_r{r:04d}.jpg")
            slicepath = str(Path(output_dir) / slicename)
            cv2.imwrite(slicepath, slice)

In [7]:
def split_dataset(params, project_dir, points_dict, trainval_dir, holdouts_dir, nolabels_dir):
    """
    This function is the main workhorse in reading the raw project folder
    and splitting it up into training, holdout, and nolabels sub-folders.
    """
    # Use the holdouts ratio defined in params.yaml to split the dataset
    holdouts_ratio = params['data']['holdouts_ratio']

    # Create the folders, if they do not already exist
    for path in [trainval_dir, holdouts_dir, nolabels_dir]:
        ensure_path(path)

    # Separate all files into empty and nonempty files, saving the former into the nolabels folder
    nonempty_filenames = []
    for filename, _ in tqdm(points_dict.items(), desc="Filtering out empty images"):
        points = points_dict[filename]
        if len(points) == 0:
            copyfile(project_dir/filename, nolabels_dir/filename)
        else:
            copyfile(project_dir/filename, trainval_dir/filename)
            nonempty_filenames.append(filename)

    # For all remaining (nonempty) files, split them up into training set and holdout set
    seed(1)
    shuffle(nonempty_filenames)
    nonempty_size = len(nonempty_filenames)
    holdouts_size = int(holdouts_ratio*nonempty_size)
    holdouts_set = nonempty_filenames[:holdouts_size]

    # This is where files are actually moved out of the trainval folder into the holdout folder
    for filename in tqdm(holdouts_set, desc="Creating holdouts set"):
        move(trainval_dir/filename, holdouts_dir/filename)

    return holdouts_set

In [8]:
def generate_training_set(params, points_dict, holdouts_set, 
                          trainval_input_dir, holdouts_input_dir,
                          trainval_output_dir, holdouts_output_dir):
    """
    Once the dataset has been separated into trainval, holdout and nolabels directories,
    this function splits each image inside the trainval and holdout directories into pieces (slices).
    Saving those slices into respective folders is done in preparation for data labelling.
    """
    # Initialise image and slice dimensions
    image_height = params['data']['image_height']
    image_width = params['data']['image_width']
    slice_height = params['slices']['height']
    slice_width = params['slices']['width']

    sl = Slicer(h=image_height, w=image_width, dh=slice_height, dw=slice_width)
    slice_dict = sl.get_slice_dict()

    trainval_sliced_dict = {}
    holdouts_sliced_dict = {}
    for img_id, coords_dict in tqdm(points_dict.items(), desc="Creating slices dictionary"):
        if img_id not in holdouts_set:
            slice_list = []
            for _, pts in coords_dict.items():
                for pt in pts:
                    slice_id = sl.get_slice_id_from_point(pt)
                    slice_list.append(slice_id)
            slice_set = set(slice_list)
            if slice_set:
                trainval_sliced_dict[img_id] = slice_set
        else:
            slice_list = []
            slice_set = set(slice_dict.keys())
            holdouts_sliced_dict[img_id] = slice_set

    _save_slices(slice_dict, trainval_sliced_dict, trainval_input_dir, trainval_output_dir, datatype='trainval')
    _save_slices(slice_dict, holdouts_sliced_dict, holdouts_input_dir, holdouts_output_dir, datatype='holdouts')

### Module execution

In [10]:
# Initialise all required variables
raw_dir = Path(params['data']['raw_dir'])
project_name = params['data']['project_name']
project_folder = raw_dir / project_name
trainval_dir = Path(params['data']['trainval_dir'])/project_name
holdouts_dir = Path(params['data']['holdouts_dir'])/project_name
nolabels_dir = Path(params['data']['nolabels_dir'])/project_name
trainval_output_dir = Path(params['slices']['trainval_dir'])/project_name/'sliced_images'
holdouts_output_dir = Path(params['slices']['holdouts_dir'])/project_name/'sliced_images'

# The .pnt file is necessary to prepare a training set.
# Otherwise the number of slices quickly becomes intractable.
points_file = [file for file in os.listdir(project_folder) if file.endswith('.pnt')][0]
points_path = project_folder / points_file
try:
    with open(points_path) as file:
        data = file.read()
        data = json.loads(data)
        points_dict = data.get('points')
except FileNotFoundError as err:
    print(f"{err}")
    raise
except NameError as err:
    print(f"{err}")
    print(f"Ensure that the .pnt file is properly formatted")
    raise

# First split the raw images into categories: trainval, holdouts, nolabels
holdouts_set = split_dataset(params, raw_dir / project_name,
                                points_dict, trainval_dir,
                                holdouts_dir, nolabels_dir)

# Then, for the trainval and holdouts datasets, split them up and save the slices
# that have been identified by the .pnt file as containing objects. This is done to
# filter out empty slices and reduce the labelling required for building a detector.
generate_training_set(params, points_dict, holdouts_set, 
                        trainval_dir, holdouts_dir,
                        trainval_output_dir, holdouts_output_dir)

Filtering out empty images: 100%|██████████| 6/6 [00:00<00:00,  7.98it/s]
Creating holdouts set: 0it [00:00, ?it/s]
Saving trainval slices: 100%|██████████| 6/6 [00:03<00:00,  1.92it/s]
Saving holdouts slices: 0it [00:00, ?it/s]


# Start label studio

Please note that this code will only run using a bash or zsh shell

In [ ]:
def start_labeling_job(trainval_dir):
    cmd = ["chmod", "+x", "./scripts/get_urls.sh"]
    run(cmd)

    cmd = ["./scripts/get_urls.sh", f"{trainval_dir}", "*.jpg"]
    Popen(cmd)

    cmd = ["label-studio", "start"]
    Popen(cmd)

# Prepare training set

This script is used to scan through labels produced from label studio 
and prepare a dataset in the COCO format required for training.

### Function definitions

In [ ]:
def save_slices(filepath, slice_ids, slice_dict, output_dir):
    """
    This function loads an image, slices it up, and saves the pieces (slices) as separate images.
    It returns the paths of each slice as a list.
    """
    img = cv2.imread(filepath)
    filename = os.path.basename(filepath)
    slicepaths = []
    for slice_id in slice_ids:
        l,t,r,b = slice_dict[slice_id]
        slice = img[t:b, l:r]
        slicename = (os.path.splitext(filename)[0] + f"_t{t:04d}_b{b:04d}_l{l:04d}_r{r:04d}.jpg")
        slicepath = str(Path(output_dir) / slicename)
        cv2.imwrite(slicepath, slice)
        slicepaths.append(slicepath)
    return slicepaths

In [ ]:
def cocofy_annotations(data_dir, labels):
    """
    This function is used to reformat the annotations from label-studio into that required for training.

    Inputs:
        - data_dir: Data directory where original slice is located
        - labels: Label-studio annotations

    Outpus:
        - annotations: Dictionary containing COCO formatted annotations
    """
    annotations = {}
    for label in labels:
        lbl_annotations = label['annotations'][0]
        if not lbl_annotations['was_cancelled']:
            # This if statement is meant to deal with issues that may arise with label studio if saving over a project
            if len(lbl_annotations['result'])==0 and lbl_annotations.get('prediction'):
                old_result = lbl_annotations['prediction']['result']
                lbl_annotations['result'] = old_result if len(old_result) > 3 else lbl_annotations['result']
            
            # Convert label studio annotations to COCO format
            old_bbox_data = lbl_annotations['result']
            coco_bbox_data = []
            for bbox in old_bbox_data:
                old_bbox = bbox['value']
                coco_bbox = [old_bbox['x']/100, old_bbox['y']/100, old_bbox['width']/100, old_bbox['height']/100]
                coco_label = old_bbox['rectanglelabels'][0]
                coco_bbox_datum = {'bbox': coco_bbox, 'label': coco_label}
                coco_bbox_data.append(coco_bbox_datum)

            filename = os.path.basename(label['data']['image'])
            annotations[str(Path(data_dir) / filename)] = coco_bbox_data
    return annotations

In [11]:
def prepare_training_set(params):
    """
    This function scans through the labels produced from label studio 
    and prepares a dataset in a format required for training

    Inputs: params (see params.yaml)

    Outputs: a dataset in COCO format located in training->train_coco (see params.yaml)
    """
    
    # Initialise required variables (see params.yaml)
    train_data_dir = ensure_path(params['training']['train_data_dir'])
    train_coco = params['training']['train_coco']
    project_name = params['data']['project_name']
    nolabels_dir = Path(params['data']['nolabels_dir'])/project_name
    slices_trainval_dir = Path(params['slices']['trainval_dir'])/project_name/'sliced_images'
    slices_nolabels_dir = Path(params['slices']['nolabels_dir'])/project_name/'sliced_images'

    image_height = params['data']['image_height']
    image_width = params['data']['image_width']
    slice_height = params['slices']['height']
    slice_width = params['slices']['width']

    # This labels variable is the filepath containing the output of the label-studio labelling job.
    labels = params['slices']['labels']

    # This function transforms, or re-formats, the labels into the required format for training.
    with open(labels, 'r') as fr:
        annotations = cocofy_annotations(slices_trainval_dir, json.loads(fr.read()))

    # This function adds the new annotations to a dataset ready for training
    coco_dataset = CocoDataset(train_data_dir)
    coco_dataset.add_existing_annotations(annotations)

    # This loop scans through the empty images (those containing no labels), slices them up, then
    # subsamples a few slices to include in the training set to reduce False Positive Rate.
    sl = Slicer(h=image_height, w=image_width, dh=slice_height, dw=slice_width)
    slice_dict = sl.get_slice_dict()
    output_dir = ensure_path(slices_nolabels_dir)
    i = 0
    for filename in tqdm(os.listdir(nolabels_dir), desc="Adding negative samples"):
        i+=1
        seed(i)
        filepath = str(Path(nolabels_dir) / filename)
        slice_ids = sample(list(slice_dict), 3)
        slicepaths = save_slices(filepath, slice_ids, slice_dict, output_dir)
        coco_dataset.add_negative_samples(slicepaths)
    
    # The dataset is exported into a format ready for training
    coco_dataset.export_dataset(train_coco)

### Module execution

In [12]:
prepare_training_set(params)

Adding existing annotations: 100%|██████████| 2/2 [00:00<00:00, 64.51it/s]
Adding negative samples: 0it [00:00, ?it/s]


# Train model

### CNN library imports

In [16]:
import gc
import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from utils.train_utils import collate_fn, get_train_transform, BirdDataset

### Function definitions

In [ ]:
def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    """
    This is the "training loop", which calculates the loss for each epoch
    """
    running_loss = 0.0
    for images, labels, filenames in tqdm(data_loader, desc="Training"):
        images = list(image.to(device) for image in images)
        labels = [{k: v.to(device) for k, v in l.items()} for l in labels]
        optimizer.zero_grad()
        loss_dict = model(images, labels)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        running_loss += losses.item()
        del images, labels
        gc.collect()
        torch.cuda.empty_cache()
    train_loss = running_loss / float(len(data_loader))
    scheduler.step(train_loss)
    return train_loss

In [ ]:
def eval_loop_fn(data_loader, model, device):
    """
    This is the "validation loop", which calculates the loss for each epoch
    """
    running_loss = 0.0
    for images, labels, filenames in tqdm(data_loader, desc="Validation"):
        images = list(image.to(device) for image in images)
        labels = [{k: v.to(device) for k, v in l.items()} for l in labels]
        loss_dict = model(images, labels)
        losses = sum(loss for loss in loss_dict.values())
        running_loss += losses.item()
        del images , labels
        gc.collect()
        torch.cuda.empty_cache()
    valid_loss = running_loss / float(len(data_loader))
    return valid_loss

In [17]:
def train_mdba(params):
    """
    This is the main function that actually performs the training.

    Inputs: params from params.yaml file

    Outputs: a best model is saved in the best_model_path
    """

    # Define training parameters, typically no need to change these
    lr = params['training']['learning_rate']
    wd = params['training']['weight_decay']
    momentum = params['training']['momentum']
    early_stopping_patience = params['training']['early_stopping_patience']
    num_epochs = params['training']['num_epochs']

    best_model_path = Path(params['models']['best_model']) # Where output model will be saved
    ensure_path(params['models']['weights_dir']) # Ensure the output model folder exists

    # Define the dataset using training transforms (see ./utils/train_utils.py)
    # NOTE: it may be useful to split train and validation transforms before this step
    my_dataset = BirdDataset(
        root=params['training']['train_data_dir'],
        annotation=params['training']['train_coco'],
        transforms=get_train_transform())

    # Split the dataset into training and validation sets
    val_ratio = params['training']['val_ratio']
    total_size = len(my_dataset)
    val_size = int(val_ratio*total_size)
    train_size = total_size-val_size
    train_dataset, valid_dataset = torch.utils.data.random_split(
        my_dataset, (train_size, val_size))

    # Definte data loaders for training and validation sets
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=params['training']['train_batch_size'],
        shuffle=True,
        num_workers=params['training']['num_workers_dl'],
        collate_fn=collate_fn)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=params['training']['valid_batch_size'],
        shuffle=False,
        num_workers=params['training']['num_workers_dl'],
        collate_fn=collate_fn)

    # Load the model into memory
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.to(device)

    # Define optimizer and scheduler
    model_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(model_params, lr=lr, momentum=momentum, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

    # Run main training loop, save a model when an epoch outperforms previous best
    all_losses = []
    for epoch in range(num_epochs):
        print(f"Epoch --> {epoch+1} / {num_epochs}")
        print(f"-------------------------------")
        train_loss = train_loop_fn(train_loader, model, optimizer, device, scheduler)
        print('training Loss: {:.4f}'.format(train_loss))
        valid_loss = eval_loop_fn(valid_loader, model, device)
        print('validation Loss: {:.4f}'.format(valid_loss))
        all_losses.append(valid_loss)
        
        ## Uncomment this line to save model after every epoch
        # torch.save(model, f"{weights_dir}/epoch-{epoch}_lr-{lr}.pth")

        if epoch == 0: # Save model as "best model" after first epoch
            torch.save(model, best_model_path)
            best_loss = valid_loss
            best_epoch = epoch
        elif valid_loss < best_loss: # Save model as "best model" if it results in lower loss
            torch.save(model, best_model_path)
            best_loss = valid_loss
            best_epoch = epoch
        elif epoch - best_epoch == early_stopping_patience: # Stop if no progress has been made
            print(f"Early stopping condition reached. Stopping training.")
            break

### Module Execution

In [18]:
train_mdba(params)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Epoch --> 1 / 10
-------------------------------


Training:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\a1629970\.conda\envs\ML\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Training: 100%|██████████| 1/1 [00:04<00:00,  4.81s/it]
c:\Users\a1629970\.conda\envs\ML\lib\site-packages\torch\optim\lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


training Loss: 0.9810


Validation: 100%|██████████| 1/1 [00:11<00:00, 11.10s/it]


validation Loss: 0.9202
Epoch --> 2 / 10
-------------------------------


...

validation Loss: 0.6110
Epoch --> 8 / 10
-------------------------------


Training: 100%|██████████| 1/1 [00:09<00:00,  9.78s/it]


training Loss: 0.2795


Validation: 100%|██████████| 1/1 [00:11<00:00, 11.57s/it]

validation Loss: 0.5962
Early stopping condition reached. Stopping training.


# Count birds with trained model

After a model has been trained to count species, this script performs the counting.
It does so by calling a function within ./utils/count.py that slices images then feeds
these slices into the model, one by one, before tallying up the counts and outputting them.

Results are saved in an output folder defined below (./model/results by default).

Necessary inputs (see params.yaml file for defaults):
<ul>
    <li>data-root: Root directory (usually ./data/raw) where projects (folders) containing raw images is stored </li>
    <li>project-name: Name of the folder where raw images are stored, which will act as inputs for prediction.</li>
    <li>image-height: This is the height (in pixels) of raw images (must be consistent across all images).</li>
    <li>image-width: This is the width (in pixels) of raw images (must be consistent across all images).</li>
    <li>slice-height: This is the height (in pixels) of individual slices. These slices are used for inference.</li>
    <li>slice-width: This is the width (in pixels) of individual slices. These slices are used for inference.</li>
    <li>overlap: Vertical overlap ratio between adjacent images. Depends on the mission.</li>
    <li>sidelap: Horizontal overlap ratio between adjacent images. Depends on the mission.</li>
    <li>categories: Categories to detect. See example function call above to deal with multiple categories.</li>
        <ul>
        <li>-- NOTE: If using customised categories (not recommended!):</li>
            <ul>
            <li>number of categories must match the number that the model was trained against</li>
            <li>categories must be a list of dictionaries with this format:</li>
            </ul>
                <ul>
                <li>- e.g. [
                    {'id': 0, 'name': 'bird', 'supercategory': 'animal'},
                    {'id': 1, 'name': 'dog', 'supercategory': 'animal'},
                ]</li>
                </ul>
        </ul>
</ul>

Optional inputs (see params.yaml file for defaults)
<ul>
    <li>output-folder: This is the folder where outputs will be stored.</li>
    <li>conf-thresh: Binary cut-off threshold for detection. Can be used to calibrate for under/overcounting.</li>
    <li>stride: Lower values increase computational time for improved performance. Must be > 0 and <=1.</li>
    <li>model-path: Model file path. Either update the params.yaml file or the default one in this function.</li>
    <li>save-slices: Setting this argument to True saves all the individual slices (inputs). Useful for analysis.</li>
</ul>

Outputs:
<ul>
    <li>Look for outputs in the value defined in params.yaml (evaluation->results_dir) or define a new location here.</li>
</ul>

### Function definition

In [ ]:
def parse_args(params):
    """
    Defining input arguments
    """

    # See params.yaml for these values located in their respective tree. 
    # e.g. model_path is the value of (models -> best_model) inside params.yaml 
    raw_dir = Path(params['data']['raw_dir'])
    project_name = params['data']['project_name']
    model_path = Path(params['models']['best_model'])
    results_dir = Path(params['evaluation']['results_dir'])
    conf_thresh = params['evaluation']['conf_thresh']
    overlap = params['evaluation']['overlap']
    sidelap = params['evaluation']['sidelap']
    stride = params['evaluation']['stride']

    # Initialise image and slice dimensions
    image_height = params['data']['image_height']
    image_width = params['data']['image_width']
    slice_height = params['slices']['height']
    slice_width = params['slices']['width']

    # This file is generated when preparing the dataset before training.
    # See prepare_training_set() function in prepare_training_set.py for details.
    with open(params['training']['train_coco'], 'r') as labels_file:
        categories = json.load(labels_file).get('categories')

    # Arguments for use in command line interaction with counting script
    parser = argparse.ArgumentParser(description='Compare predicted vs true counts')
    parser.add_argument('--data-root', type=str,
                        default=raw_dir, help="Root of the data directory (default: ./data/raw/)")
    parser.add_argument('--project-name', type=str,
                        default=project_name, help='Project (image) directory')
    parser.add_argument('--image-height', type=int,
                        default=image_height, help='Height of raw images.')
    parser.add_argument('--image-width', type=int,
                        default=image_width, help='Width of raw images.')
    parser.add_argument('--slice-height', type=int,
                        default=slice_height, help='Height of slices.')
    parser.add_argument('--slice-width', type=int,
                        default=slice_width, help='Width of slices.')
    parser.add_argument('--output-folder', type=str,
                        default=results_dir, help='Folder to output results')
    parser.add_argument('--conf-thresh', type=float,
                        default=conf_thresh, help='Binary (cut-off) confidence threshold (default: 0.95)')
    parser.add_argument('--overlap', type=float,
                        default=overlap, help='Overlap ratio from 0 to 1 (default: 0.1)')
    parser.add_argument('--sidelap', type=float,
                        default=sidelap, help='Sidelap ratio from 0 to 1 (default: 0.1)')
    parser.add_argument('--stride', type=float,
                        default=stride, help='Stride (or overlap) ratio between slices (top and bottom)')
    parser.add_argument('--model-path', type=str,
                        default=model_path, help='Path to model (default: ./models/detection/weights/best.pth)')
    parser.add_argument('--categories', type=list,
                        default=categories, help='Categories from labels.json')
    parser.add_argument('--save-slices', type=str2bool,
                        default=False, help='Save slices (default: False). Note: Only set to true when debugging')
    args = parser.parse_args()

    return args




### Module execution

In [ ]:
args = parse_args(params)

# Execute counting script, located in utils/count.py
count_folder(args)

# Evaluate trained model

If ground truth counts are provided, this module performs linear regression using predicted and manual counts as input

There are two outputs:
<ul>
    <li> A csv containing the predicted and manual counts </li>
    <li> A .png figure of the linear regression labelled with the equation and r-squared</li>
</ul>

In [ ]:
def eval_mdba(params):
    """
    This function takes the project name in params.yaml as a parameter, then runs
    an inference over all the slices (images) in the holdout set for that project.
    """

    # Initialise inputs
    project_name = params['data']['project_name']
    slice_holdout_dir = Path(params['slices']['holdouts_dir'])/project_name/'sliced_images'
    results_dir = ensure_path(Path(params['evaluation']['results_dir']))
    conf_thresh = params['evaluation']['conf_thresh']

    # Load model
    model = torch.load(params['models']['best_model'])
    model.cuda()
    model.eval()
    transform = get_validation_transform()

    # For each file in the holdout set, make a prediction and 
    # store the outputs in a dictionary called predictions
    predictions = {}
    filenames = sorted(os.listdir(slice_holdout_dir))
    img_ids = []
    first=True
    for filename in tqdm(filenames, desc="Counting birds in holdout set"):
        img_id = '_'.join(filename.split('_')[:3]) + '.JPG'
        if predictions.get(img_id) is None:
            predictions[img_id] = 0
            img_ids.append(img_id)
            if not first:
                print(f"Count for {prev_img_id} == {predictions[prev_img_id]}")
            first=False
        filepath = Path(slice_holdout_dir)/filename
        image = Image.open(filepath)
        image = transform(image).unsqueeze(0).cuda()

        # This line is where the actual "inference" occurs.
        # Write functionality after this line for customised analysis.
        # This variable is a dictionary with three keys: "boxes", "labels" and "scores".
        out = model(image)[0]

        scores = out['scores'].cpu().detach().numpy()
        predictions[img_id] += len(scores[scores>conf_thresh])
        prev_img_id = img_id
    
    print(f"Count for {prev_img_id} == {predictions[prev_img_id]}")
    ground_truth = {}
    ground_truth_data = params['data']['ground_truth_counts']
    with open(ground_truth_data, "r") as csv_file:
        gt_data = csv.reader(csv_file)
        for row in gt_data:
            if row[0] in img_ids:
                ground_truth[row[0]] = int(row[1])

    xs = []
    ys = []
    for key, value in predictions.items():
        xs.append(ground_truth[key])
        ys.append(value)

    xs = np.array(xs)
    ys = np.array(ys)

    xs_fit = xs[:,np.newaxis]
    a, res, _, _ = np.linalg.lstsq(xs_fit, ys, rcond=None)
    r2 = 1 - res / (ys.size * ys.var())

    # Output CSV and PNG
    output_basename = "true_vs_predicted_counts"
    dataset = pd.DataFrame({'MANUAL': xs, 'AUTOMATIC': ys}, columns=['MANUAL', 'AUTOMATIC'])
    dataset.to_csv(f"{results_dir}/{output_basename}.csv")

    plt.rcParams["figure.figsize"] = (20,10)
    plt.rcParams.update({'font.size': 25})
    pylab.plot(xs,ys,'o')
    pylab.plot(xs,a*xs,"r--")
    plt.xlim(0.9,2200)
    plt.ylim(0.9,2200) 
    plt.ylabel('Predicted Count')
    plt.xlabel('True Count')
    plt.title('Comparing Ground Truth and Predicted Counts (Holdout Set)')
    plt.text(650, 750, f"y={a[0]:.3f}x," + " $\mathregular{R^{2}}$" + f"= {r2[0]:0.3f}", rotation = 27.5)
    plt.savefig(f"{results_dir}/{output_basename}.png")

### Module execution

In [ ]:
eval_mdba(params)